## Toronto Rental Listings Capstone Project

### Inputting the libraries we will be using. 

In [1]:
#Capstone Project 

import requests 

import numpy as np 

from bs4 import BeautifulSoup

import pandas as pd # library for data analsysis
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

## Inputting Toronto Neighbourhood Data 

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

df_tor = df[0]

#Ignoring data entries where Borough's are 'Not assigned'

df_tor = df_tor[df_tor.Borough != 'Not assigned'].reset_index(drop=True)

In [4]:
df_tor2 = df_tor.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

df_tor2.columns = ['Postcode', 'Borough', 'Neighbourhood']

df_tor2['Neighbourhood']=df_tor2['Neighbourhood'].replace('Not assigned', df_tor2['Borough']) #Replacing not assigned Neighbourhood with Borough value
df_tor2.rename(columns= {'Postcode': 'PostalCode'}, inplace =True)


#Part 2 
#!conda install -c conda-forge geocoder -y

## Inputting Geospatial data for Toronto 

In [5]:
#Import the csv file with the coordinates

geo_link= "https://cocl.us/Geospatial_data"

df_coords = pd.read_csv(geo_link)

df_coords.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)


#Merge the lattitude and longitude to our original data frame
df_tor3 = pd.merge(df_tor2, df_coords,left_on = 'PostalCode', right_on = 'PostalCode')
df_tor3

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [7]:
#Selecting the Boroughs that only contain "Downtown Toronto so that only the downtown toronto neighbourhoods can be selected"
df_to = df_tor3[df_tor3['Borough'].str.contains('Downtown Toronto')]
df_to

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
54,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
58,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
59,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


## Coordinates for University of Toronto 

In [8]:
#Obtained the geographical coordinates of UofT via https://latitude.to/articles-by-country/ca/canada/986/university-of-toronto 

latitude = 43.656997372 
longitude = -79.390331772
print('The geograpical coordinates of University of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of University of Toronto are 43.656997372, -79.390331772.


## Map of Downtown Toronto neighbourhoods 

In [9]:
#Folium map of the Downtown Toronto Neighbourhoods (19 of them)

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_to['Latitude'], df_to['Longitude'], df_to['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto



In [10]:
CLIENT_ID = 'FSPFJDZRENCOMJZMT4TFMY3P12A2KL0TTNFIOXY1FD3IYWAG' # your Foursquare ID
CLIENT_SECRET = 'QCWGJQLSOFFB4BGAXIOFW5KCSW53TSEJ12EOS4JS0OK2RRGN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FSPFJDZRENCOMJZMT4TFMY3P12A2KL0TTNFIOXY1FD3IYWAG
CLIENT_SECRET:QCWGJQLSOFFB4BGAXIOFW5KCSW53TSEJ12EOS4JS0OK2RRGN


In [11]:
#Created new df with Downtown Toronto Neighbourhoods. 
df_tor3 = df_to.reset_index(drop=True)

neighborhood_latitude = df_tor3.loc[0, 'Latitude'] 
neighborhood_longitude = df_tor3.loc[0, 'Longitude'] 

neighborhood_name = df_tor3.loc[0, 'Neighbourhood'] 

#Getting the coordinates of the first neighbourhood in downtown Toronto (Rosedale)
print('The Latitude and longitude values of the neighbourhood {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



The Latitude and longitude values of the neighbourhood Rosedale are 43.6795626, -79.37752940000001.


In [12]:
#Finding top 25 venues near Rosedale's neighbourhood using Foursquare API (similar to Week 3)

LIMIT = 25 #Number of venues (via foursquare API)
radius = 1000 
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [13]:
results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filtering the columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# cleaning the columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues #Nearby venues near Rosedale neighbourhood 

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
2,Black Camel,BBQ Joint,43.677016,-79.389367
3,Craigleigh Gardens,Park,43.678099,-79.371586
4,Maison Selby,Bistro,43.671232,-79.376618
5,Tinuno,Filipino Restaurant,43.671281,-79.374920
6,Pie Squared,Pie Shop,43.672143,-79.377856
7,Starbucks,Coffee Shop,43.671082,-79.380756
8,Booster Juice,Smoothie Shop,43.671566,-79.378581
9,No Frills,Grocery Store,43.671616,-79.378187


## Nearby Venues 

In [15]:
#Nearby venues for the remaining downtown Toronto neighbourhoods. 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL for connecting with fourquare 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=df_tor3['Neighbourhood'],
                                   latitudes=df_tor3['Latitude'],
                                   longitudes=df_tor3['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Queen's Park


In [17]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",25,25,25,25,25,25
Berczy Park,25,25,25,25,25,25
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",25,25,25,25,25,25
Central Bay Street,25,25,25,25,25,25
"Chinatown,Grange Park,Kensington Market",25,25,25,25,25,25
Christie,17,17,17,17,17,17
Church and Wellesley,25,25,25,25,25,25
"Commerce Court,Victoria Hotel",25,25,25,25,25,25


In [18]:
#We want to see the top venues in each downtown neighbourhood.  

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape
toronto_onehot.head()

,Wine Bar,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    #We can see the top 5 venues in each neighbourhood, as well as the frequency of the respective ones below.

----Adelaide,King,Richmond----
                  venue  freq
0                 Hotel  0.08
1      Asian Restaurant  0.08
2            Steakhouse  0.08
3  Gym / Fitness Center  0.04
4      Sushi Restaurant  0.04
5           Pizza Place  0.04
6           Coffee Shop  0.04
7           Opera House  0.04
8                 Plaza  0.04
9                  Café  0.04


----Berczy Park----
                venue  freq
0      Farmers Market  0.08
1                Café  0.08
2  Seafood Restaurant  0.08
3            Beer Bar  0.08
4  Italian Restaurant  0.04
5  Basketball Stadium  0.04
6                Park  0.04
7         Coffee Shop  0.04
8        Cocktail Bar  0.04
9         Fish Market  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0      Airport Lounge  0.13
1     Airport Service  0.13
2    Airport Terminal  0.13
3                 Bar  0.07
4  Airport Food Court  0.07
5        Airport Gate  0.07
6 

## Top 10 venues in each neighbourhood 

In [29]:
#Finding the top 10 types of venues in the respective neighbourhoods of downtown Toronto. 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10 #Top 10 venues 

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
#This information allows customers to see which neighbourhoods has venues that desire them the most (such as resturants in this case)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Hotel,Asian Restaurant,Steakhouse,Gym / Fitness Center,Speakeasy,Monument / Landmark,Café,Opera House,Coffee Shop,Pizza Place
1,Berczy Park,Café,Beer Bar,Seafood Restaurant,Farmers Market,Fish Market,Basketball Stadium,Concert Hall,Breakfast Spot,Italian Restaurant,Steakhouse
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Rental Car Location
3,"Cabbagetown,St. James Town",Restaurant,Café,Italian Restaurant,Market,Pub,Butcher,Pet Store,Caribbean Restaurant,Coffee Shop,Park
4,Central Bay Street,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Spa,Park,Café,Modern European Restaurant,Miscellaneous Shop,Ramen Restaurant,Sandwich Place


## K-Means Clustering 

In [27]:
#Create map of clusters for the downtown Toronto Neighbourhoods (similar to week 3)

df_tor4 = df_tor3.groupby('Neighbourhood').mean().reset_index()
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels ', kmeans.labels_) #Don't forget the extra space in 'Cluster Labels '

toronto_merged = df_tor3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how = 'right')

toronto_merged.head()

ValueError: cannot insert Cluster Labels , already exists

## Inputting Toronto Listings Data 

In [22]:
tor_listing = pd.read_csv('https://raw.githubusercontent.com/vselva06/Coursera_Capstone/master/capstone_rent_listings.csv')#Have to pass the RAW link from Github
tor_listing

,Address,Postal Code,Price,Latitude,Longitude
0,486 Clinton Street,M6G 2Z4,2600.0,43.667381,-79.418765
1,181 Dundas St East,M5A 1Z4,2500.0,43.657034,-79.374045
2,321 Sherbourne St,M5A 2S4,2170.0,43.661899,-79.372021
3,1080 Bay St,M5S 0A5,3950.0,43.666991,-79.388917
4,55 Bloor Street East,M4W 1A9,2700.0,43.670203,-79.385101
5,411 Markham St,M6G 2L1,2250.0,43.660245,-79.410382
6,887 Bay St,M5S 3K4,850.0,43.663020,-79.386136
7,152 St Patrick St,M5T 3J9,2450.0,43.654145,-79.390524
8,NaN,NaN,NaN,NaN,NaN


## Rental listings coordinates 

In [23]:
#Rental Properties         
rent1 = [43.667381, -79.418765]
rent2 = [43.657034, -79.374045]
rent3 = [43.661899, -79.372021]
rent4 = [43.666991, -79.388917]
rent5 = [43.670203, -79.385101]
rent6 = [43.660245, -79.410382]
rent7 = [43.663020, -79.386136]
rent8 = [43.654145, -79.390524]

## Map of Toronto neighbourhoods with venue clusters 

In [24]:
#Creating cluster map for VENUES 

# Creating map of clusters 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels ']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True) #Represents the cluster of venues. 
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1], #convert
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
for lat, lng, label in zip(df_to['Latitude'], df_to['Longitude'], df_to['Neighbourhood']): #For the 19 downtown neighbourhoods. 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)  
map_clusters

## Combined map of Toronto neighbourhoods with venue clusters and rental listings 

In [25]:

newmap_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

folium.CircleMarker(
        rent1,
        radius=5,
        popup="Rental 1",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)

folium.CircleMarker(
        rent2,
        radius=5,
        popup="Rental 2",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)

folium.CircleMarker(
        rent3,
        radius=5,
        popup="Rental 3",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)

folium.CircleMarker(
        rent4,
        radius=5,
        popup="Rental 4",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)

folium.CircleMarker(
        rent5,
        radius=5,
        popup="Rental 5",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)
folium.CircleMarker(
        rent6,
        radius=5,
        popup="Rental 6",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)
folium.CircleMarker(
        rent7,
        radius=5,
        popup="Rental 7",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)
folium.CircleMarker(
        rent8,
        radius=5,
        popup="Rental 8",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels ']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True) #Represents the cluster of venues. 
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1], #convert
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(newmap_toronto)
for lat, lng, label in zip(df_to['Latitude'], df_to['Longitude'], df_to['Neighbourhood']): #For the 19 downtown neighbourhoods. 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newmap_toronto)
newmap_toronto



